In [1]:
import torch
import numpy as np

# 함수 미분

In [2]:
# np를 이용한 함수
def my_derivative(t):
    return (4*t**3 + t**4 + np.cos(np.exp(t)))*np.exp(t)

In [3]:
my_derivative(1.0)

11.113059409339991

In [4]:
# torch를 이용한 함수와 auto derivative
x = torch.tensor([1.], requires_grad=True) # tenor안에 실수형 (int 오류남)
y = torch.exp(x)

y.backward()
x.grad

tensor([2.7183])

- 미분하는 문자: 대입할 값을 미리 지정해줘야 하고, `requires_grad = True`를 꼭 지정해줘야 이 문자로 미분할 것이라고 알려주는 것.
- y는 x에 연결되어있는 torch 객체로, x의 연산 기능을 그대로 가져오게된다.

> `.backward()`를 호출하면,
> - 출력 텐서에서 시작하여 입력 방향으로 그래디언트를 계산한다.
> - 계산 그래프를 따라 각 연산의 편미분을 계산하고, 체인 룰을 적용하여 전체 그래디언트를 구한다.
> - 계산된 그래디언트는 `requires_grad=True`로 설정된 각 텐서의 `.grad` 속성에 누적된다.

> `x.grad`에 값이 저장되는 원리
> - `backward()` 호출 시, PyTorch는 손실 함수에서 x까지의 모든 경로를 따라 편미분을 계산한다.
> - PyTorch의 autograd 엔진이 각 연산의 그래디언트를 자동으로 계산한다.
> - 계산된 최종 그래디언트는 `x.grad`에 저장한다. (함수를 x에 대해 미분한 결과값)
> -  그래디언트는 누적되기 때문에, `.backward()`를 여러번 호출할 경우, `.zero_()`를 호출해 x의 그래디언트 값들을 0으로 초기화햐야한다.

In [5]:
# 자동으로 체인룰을 적용하여 계산
t = torch.tensor([1.0], requires_grad=True)
u = [t**2, torch.exp(t)]
z = u[0]**2 * u[1] + torch.sin(u[1])

z.backward()
t.grad

tensor([11.1131])

In [6]:
t = torch.tensor([1.0], requires_grad=True)
u = torch.stack([t**2, torch.exp(t)])
z = u[0]**2 * u[1] + torch.sin(u[1])

z.backward()
t.grad

tensor([11.1131])

- `torch.stack()`메소드를 사용하면 차원을 유지한 채로 결합할 수 있다.
    - 새로운 차원을 생성하여 그 차원을 따라 텐서들을 쌓는다.

> - 모든 입력 텐서의 shape이 정확히 동일해야 한다.
> - 결과 텐서의 차원 수는 입력 텐서들보다 1만큼 더 많아진다.

- `.concatenate()`메서드는 입력된 기존 tensor들의 차원을 따라서 결합하기 때문에 $X$와 $Y$를 $[20,1]$차원으로 만든 후에 결합해야한다.

> - 모든 입력 텐서들의 차원이 동일해야한다.
> - 결과 텐서의 차원 수는 입력 텐서들과 동일하다.

In [9]:
t = torch.tensor([1.0], requires_grad=True)

def g(T):
    return torch.stack([T**2, torch.exp(T)])

def f(U):
    return U[0]**2 * U[1] + torch.sin(U[1])

z = f(g(t))
z.backward()
t.grad

tensor([11.1131])

In [15]:
# 행렬로 계산
u = np.array([1,2])
Dg = np.array([[1,1],
               [u[1], u[0]]])

x = np.array([u[0]+u[1], u[0]*u[1]])
Df = np.array([[2*x[0]*x[1], x[0]**2],
               [x[1]*np.cos(x[0]*x[1]), x[0]*np.cos(x[0]*x[1])]])
Df@Dg

array([[30.        , 21.        ],
       [ 7.68136229,  4.80085143]])

In [19]:
# torch로 계산
u = torch.tensor([1.,2.], requires_grad=True)
x = torch.stack([u[0]+u[1], u[0]*u[1]]) # stack으로 쌓으면 차원 유지 가능
z1 = x[0]**2 *x[1]
z1.backward()
u.grad

tensor([30., 21.])

In [21]:
u = torch.tensor([1.,2.], requires_grad=True)
x = torch.stack([u[0]+u[1], u[0]*u[1]])
z2 = torch.sin(x[0]*x[1])
z2.backward()
u.grad

tensor([7.6814, 4.8009])

스칼라값이 아닌 벡터에 대해 `.backward()` 할 경우 다음과 같은 에러 발생

```Error: grad can be implicitly created only for scalar outputs```

> `torch`의 `backward()` 메서드의 제약 사항 두가지가 있다.
> 1. **스칼라값(단일 요소를 가진 텐서)**에 대해서만 호출할 수 있다.
> 2. 출력이 스칼라가 아닌 경우, **gradient 인자를 명시적으로 제공**해야 한다. 즉, 각 출력 요소에 대한 기울기의 공간을 제시해주어야 한다.

- 1. 조건을 만족시켜 해결하기 위해, y(함수)에 `.sum()`이나 `.mean()` 등의 함수를 적용해 스칼라값으로 만들어준 뒤에 backward()한다.
- 2. 조건을 만족시켜 해결하기 위해, 다음과 같이 backward 메서드에 gradient 인자를 명시적으로 제공한다.
```
y.backward(gradient=torch.ones_like(y))
```
> `torch`의 `backward()` 메서드의 제약 사항 두가지가 있다.
> 1. **스칼라값(단일 요소를 가진 텐서)**에 대해서만 호출할 수 있다.
> 2. 출력이 스칼라가 아닌 경우, **gradient 인자를 명시적으로 제공**해야 한다. 즉, 각 출력 요소에 대한 기울기의 공간을 제시해주어야 한다.

- 1. 조건을 만족시켜 해결하기 위해, 이 문제의 경우에는 두 함수에 대해 따로따로 미분해준다. (위의 코드블록)
- 2. 조건을 만족시켜 해결하기 위해, 다음과 같이 backward 메서드에 gradient 인자를 명시적으로 제공한다.

In [16]:
u = torch.tensor([1.,2.], requires_grad=True)
x = torch.stack([u[0]+u[1], u[0]*u[1]]) # stack으로 쌓으면 차원 유지 가능
f = torch.stack([x[0]**2 *x[1], torch.sin(x[0]*x[1])])
f.backward(gradient=torch.ones_like(f))
u.grad

tensor([37.6814, 25.8008])

In [14]:
x = torch.stack([u[0]+u[1], u[0]*u[1]]) # stack으로 쌓으면 차원 유지 가능
z1 = x[0]**2 *x[1]
z2 = torch.sin(x[0]*x[1])
z2.backward()
u.grad

tensor([15.3627,  9.6017])

- 이런식으로 backward를 여러번 하면 `u.grad`에 계속 값을 누적함

In [22]:
# 해결 방법
x = torch.stack([u[0]+u[1], u[0]*u[1]]) # stack으로 쌓으면 차원 안변함
z1 = x[0]**2 *x[1]
z2 = torch.sin(x[0]*x[1])
z2.zero_() # 그래디언트 초기화
z2.backward()
u.grad

tensor([7.6814, 4.8009])

# 차원 문제 (broadcasting 이슈)

In [24]:
X=torch.tensor([-3.0000e+00, -2.7000e+00, -2.4000e+00, -2.1000e+00, -1.8000e+00,
        -1.5000e+00, -1.2000e+00, -9.0000e-01, -6.0000e-01, -3.0000e-01,
        -2.3842e-08,  3.0000e-01,  6.0000e-01,  9.0000e-01,  1.2000e+00,
         1.5000e+00,  1.8000e+00,  2.1000e+00,  2.4000e+00,  2.7000e+00])
Y = torch.tensor([-7.1452, -5.4253, -5.1977, -3.6225, -3.8022, -4.4101, -4.6622, -3.1932,
        -1.7325, -1.8879, -1.0742, -0.2320,  1.8226,  1.5453, -1.5535,  0.8857,
         1.7537,  3.1607,  1.8912,  4.0895])

In [29]:
BETA = torch.tensor([[1.0], [0.5]], requires_grad=True) # shape: [2,1]
ones = torch.ones([20])
X_ = torch.stack([ones, X], axis=1) # shape: [20,2]
Y_ = Y.reshape(-1,1) # shape: [20,1] (Y: [20])
L = torch.sum((Y_ - X_ @ BETA)**2) # Y로 하면 잘못 계산됨
L.backward()
BETA.grad

tensor([[  94.5796],
        [-159.6058]])

In [30]:
beta = torch.tensor([1.0, 0.5], requires_grad=True) # shape: [2]
beta_ = beta.reshape(2,-1) # shape: [2,1]
ones = torch.ones([20])
X_ = torch.stack([ones, X], axis=1) # shape: [20,2]
Y_ = Y.reshape(-1,1) # shape: [20,1] (Y: [20])
L = torch.sum((Y_ - X_ @ beta_)**2) # beta로 하면 잘못 계산됨
L.zero_()
L.backward()
BETA.grad

tensor([[  94.5796],
        [-159.6058]])

# 기타

- Matplotlib의 plt에서 허용되는 데이터 형식
    - `np.array`, `list`, `pandas.Series`, `pandas.DataFrame`
    - `torch.Tensor`는 허용되지 않는다

- `detach()` 메소드는 현재 Tensor 객체와 동일한 데이터를 가지지만 연산에서 분리된 새로운 Tensor 객체를 생성한다. 일반적으로 Tensor 객체를 다른 Tensor 객체로 변환하고자 할 때 사용된다.
-  `requires_grad=True`를 설정한 torch.Tensor은 연산이 포함되어있다. 그래서 그냥은 numpy로 변환할 수 없기 때문에 `detach()`를 꼭 해줘야한다.